In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, pyautogui, re, json
from bs4 import BeautifulSoup
from datetime import datetime
from winotify import Notification

In [2]:
driver = webdriver.Chrome()
driver.get("https://lactalis-smartit.onbmc.com/smartit/app/#/")
time.sleep(3)
pyautogui.hotkey("Esc")

In [3]:
def Verifica(tipo, valor:str):
    return WebDriverWait(driver, 10).until(EC.visibility_of_element_located((tipo, valor)))

In [4]:
Verifica(By.XPATH, '//*[@id="main"]/div/div[1]/div[1]/div/div/button').click()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF672AAEEB2+31554]
	(No symbol) [0x00007FF672A27EE9]
	(No symbol) [0x00007FF6728E872A]
	(No symbol) [0x00007FF672938434]
	(No symbol) [0x00007FF67293853C]
	(No symbol) [0x00007FF67297F6A7]
	(No symbol) [0x00007FF67295D06F]
	(No symbol) [0x00007FF67297C977]
	(No symbol) [0x00007FF67295CDD3]
	(No symbol) [0x00007FF67292A33B]
	(No symbol) [0x00007FF67292AED1]
	GetHandleVerifier [0x00007FF672DB8B2D+3217341]
	GetHandleVerifier [0x00007FF672E05AF3+3532675]
	GetHandleVerifier [0x00007FF672DFB0F0+3489152]
	GetHandleVerifier [0x00007FF672B5E786+750614]
	(No symbol) [0x00007FF672A3376F]
	(No symbol) [0x00007FF672A2EB24]
	(No symbol) [0x00007FF672A2ECB2]
	(No symbol) [0x00007FF672A1E17F]
	BaseThreadInitThunk [0x00007FF93A0A257D+29]
	RtlUserThreadStart [0x00007FF93B92AF28+40]


In [5]:
Verifica(By.NAME, "AUTHENTICATION.LOGIN").send_keys("joaquim.junior.terceiro@br.lactalis.com")
driver.find_element(By.NAME, "AUTHENTICATION.PASSWORD").send_keys("L@ctali_S2024")
driver.find_element(By.NAME, "validateButton").click()

In [6]:
Verifica(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div/button').click()
time.sleep(2)
total_chamados = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div[4]/h3/span').text
blocos_total = int(total_chamados)//75

In [7]:
def list_chamados(html):
    soup = BeautifulSoup(html, 'html.parser')
    chamados = soup.findAll('span', class_=re.compile('ng-binding ng-scope'))[7:]
    chamados_geral = []
    for chamado in chamados:
        chamados_geral.append(chamado.getText())
    return chamados_geral

def ResetBlocos(blocos_total):
    for bloco in range(0, blocos_total):
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()

In [10]:
chamados_tel = []
for bloco in range(0, blocos_total+1):
    if bloco == 0:
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text == 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text != 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button[2]').click()
        chamados_tel.append(list_chamados(driver.page_source))
ResetBlocos(blocos_total)

In [ ]:
chamados_tel = []
for bloco in range(0, blocos_total+1):
    if bloco == 0:
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text == 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text != 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button[2]').click()
        chamados_tel.append(list_chamados(driver.page_source))
ResetBlocos(blocos_total)

In [11]:
dados_chamados_geral = []

for n in range(0, len(chamados_tel)):
    for chamado in chamados_tel[n][7::10]:
        dados_dict = {}
        for item in chamado.split('\n- ')[1:-5]:
            try:
                chave, valor = item.split(':', 1)
                dados_dict[chave.strip()] = valor.strip()
            except:
                chave, valor = item.split('  ', 1)
                dados_dict[chave.strip()] = valor.strip()
                
        
        # Converter o dicionário para JSON
        dados_json = json.dumps(dados_dict, indent=4, ensure_ascii=False)
        dados_chamados_geral.append(dados_dict)
        
pd.DataFrame(dados_chamados_geral)


,Nome do solicitante,Solicitado para,Site (Filial),Diretoria,Empresa,Pedido,Q_Cat_Ope,Q_Support_Group,Descreva sua solicitação,Telefone para contato,...,Descrição,ID,Localidade/Unidade,Telefone de contato,Organização (empresa),Departamento,Processo,Frente,Nome completo,Descreva o problema
0,MILENA DE ANDRADE,MILENA DE ANDRADE,BRA-026-HUNGRIA,Supply,: Lactalis,Nova Linha,Request,Mobile Phone - BR,: Chip corporativo,11912573181,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lucas Dutra Koehler,Lucas Dutra Koehler,BRA-053-TRES DE MAIO,Industrial,: Lactalis,Novo aparelho,Request,Mobile Phone - BR,: Solicito telefone móvel com linha.,55 3535 9818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WELINGTON NUNES CHALEGRE,WELINGTON NUNES CHALEGRE,BRA-042-BARRA DA TIJUCA,Comercial,: Lactalis,Novo aparelho,Request,Mobile Phone - BR,: Preciso realizar a troca do meu aparelho cel...,21-995295410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JULIANA OLIVEIRA MEDEIROS,JULIANA OLIVEIRA MEDEIROS,BRA-037-PORTO ALEGRE,Comercial,: Lactalis,Novo aparelho,Request,Mobile Phone - BR,: NOVO APARELHO + LINHA,47 8828-6763,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Marisangela Rocha,Marisangela Rocha,BRA-026-HUNGRIA,Comercial,: Lactalis,Novo aparelho,Request,Mobile Phone - BR,: NOVO SUPERVISOR COMERCIAL SP_CO (AUMENTO DE ...,11965594642,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,Cristiano Bauer de Oliveira,WAGNER DE SOUZA TEIXEIRA,BRA-071-BELO HORIZONTE,Industrial,: Itambé,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,ASCHILEY CAMILA DE AZEVEDO,ASCHILEY CAMILA DE AZEVEDO,BRA-081-SAO PAULO,Comercial,: Itambé,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535,MARCIO ROBERTO ALVES,MARCIO ROBERTO ALVES,BRA-078-PARA DE MINAS,Industrial,: Itambé,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,Lucas Wisnieski,Lucas Wisnieski,BRA-026-HUNGRIA,NaN,Lactalis,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Problemas Técnicos (Modulos SAP),Comercial Logistico (SD / LES ),Lucas Wisnieski,Erro na determinação do IVA na criação do pedi...


In [12]:
grupo_chamado = []
geral_chamado = []
status_chamado = []
tipo_chamado = []
designado_chamado = []
filial_chamado = []
cliente_chamado = []
criacao_chamado = []
n_chamado = []

for tel in chamados_tel:
    grupo_chamado.extend(tel[8::10])
    geral_chamado.extend(tel[7::10])
    status_chamado.extend(tel[6::10])
    tipo_chamado.extend(tel[5::10])
    designado_chamado.extend(tel[4::10])
    filial_chamado.extend(tel[3::10])
    cliente_chamado.extend(tel[2::10])
    criacao_chamado.extend(tel[1::10])
    n_chamado.extend(tel[::10])
    
df = pd.DataFrame({
    'Número do Chamado': n_chamado,
    'Cliente': cliente_chamado,
    'Filial': filial_chamado,
    'Grupo designado': grupo_chamado,
    'Tipo': tipo_chamado,
    'Status': status_chamado,
    'Data de criação': criacao_chamado,
})
 
df

,Número do Chamado,Cliente,Filial,Grupo designado,Tipo,Status,Data de criação
0,WO0000000346738,MILENA DE ANDRADE,BRA-026-HUNGRIA,Mobile Phone - BR,845985 - BR_20 - Solicitação Telefonia Celular,Designado,25 de jul de 2024 15:32:34
1,WO0000000346059,Lucas Dutra Koehler,BRA-053-TRES DE MAIO,Mobile Phone - BR,841096 - BR_20 - Solicitação Telefonia Celular,Designado,24 de jul de 2024 10:52:57
2,WO0000000345784,WELINGTON NUNES CHALEGRE,BRA-042-BARRA DA TIJUCA,Mobile Phone - BR,838233 - BR_20 - Solicitação Telefonia Celular,Designado,23 de jul de 2024 16:44:08
3,WO0000000345719,JULIANA OLIVEIRA MEDEIROS,BRA-037-PORTO ALEGRE,Mobile Phone - BR,841576 - BR_20 - Solicitação Telefonia Celular,Designado,23 de jul de 2024 15:27:55
4,WO0000000344532,Marisangela Rocha,BRA-026-HUNGRIA,Mobile Phone - BR,841395 - BR_20 - Solicitação Telefonia Celular,Em Andamento,19 de jul de 2024 00:10:01
...,...,...,...,...,...,...,...
533,INC000001010160,WAGNER DE SOUZA TEIXEIRA,BRA-082-SETE LAGOAS,Mobile Phone ITB - BR,BR_21 - Problema Telefonia Celular,Pendente,7 de mai de 2024 14:33:36
534,INC000001010037,ASCHILEY CAMILA DE AZEVEDO,BRA-081-SAO PAULO,Mobile Phone ITB - BR,BR_21 - Problema Telefonia Celular,Pendente,7 de mai de 2024 11:11:09
535,INC000001008022,MARCIO ROBERTO ALVES,BRA-078-PARA DE MINAS,Mobile Phone ITB - BR,BR_21 - Problema Telefonia Celular,Pendente,2 de mai de 2024 09:33:45
536,INC000000986933,Lucas Wisnieski,BRA-026-HUNGRIA,NEWCO_CONSULTORIA_COMPRAS - BR,BR_2024 - Newco,Pendente,11 de mar de 2024 17:42:20


In [13]:
df_full = pd.concat([df, pd.DataFrame(dados_chamados_geral)], axis=1)
df_full

,Número do Chamado,Cliente,Filial,Grupo designado,Tipo,Status,Data de criação,Nome do solicitante,Solicitado para,Site (Filial),...,Descrição,ID,Localidade/Unidade,Telefone de contato,Organização (empresa),Departamento,Processo,Frente,Nome completo,Descreva o problema
0,WO0000000346738,MILENA DE ANDRADE,BRA-026-HUNGRIA,Mobile Phone - BR,845985 - BR_20 - Solicitação Telefonia Celular,Designado,25 de jul de 2024 15:32:34,MILENA DE ANDRADE,MILENA DE ANDRADE,BRA-026-HUNGRIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WO0000000346059,Lucas Dutra Koehler,BRA-053-TRES DE MAIO,Mobile Phone - BR,841096 - BR_20 - Solicitação Telefonia Celular,Designado,24 de jul de 2024 10:52:57,Lucas Dutra Koehler,Lucas Dutra Koehler,BRA-053-TRES DE MAIO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WO0000000345784,WELINGTON NUNES CHALEGRE,BRA-042-BARRA DA TIJUCA,Mobile Phone - BR,838233 - BR_20 - Solicitação Telefonia Celular,Designado,23 de jul de 2024 16:44:08,WELINGTON NUNES CHALEGRE,WELINGTON NUNES CHALEGRE,BRA-042-BARRA DA TIJUCA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WO0000000345719,JULIANA OLIVEIRA MEDEIROS,BRA-037-PORTO ALEGRE,Mobile Phone - BR,841576 - BR_20 - Solicitação Telefonia Celular,Designado,23 de jul de 2024 15:27:55,JULIANA OLIVEIRA MEDEIROS,JULIANA OLIVEIRA MEDEIROS,BRA-037-PORTO ALEGRE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WO0000000344532,Marisangela Rocha,BRA-026-HUNGRIA,Mobile Phone - BR,841395 - BR_20 - Solicitação Telefonia Celular,Em Andamento,19 de jul de 2024 00:10:01,Marisangela Rocha,Marisangela Rocha,BRA-026-HUNGRIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,INC000001010160,WAGNER DE SOUZA TEIXEIRA,BRA-082-SETE LAGOAS,Mobile Phone ITB - BR,BR_21 - Problema Telefonia Celular,Pendente,7 de mai de 2024 14:33:36,Cristiano Bauer de Oliveira,WAGNER DE SOUZA TEIXEIRA,BRA-071-BELO HORIZONTE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,INC000001010037,ASCHILEY CAMILA DE AZEVEDO,BRA-081-SAO PAULO,Mobile Phone ITB - BR,BR_21 - Problema Telefonia Celular,Pendente,7 de mai de 2024 11:11:09,ASCHILEY CAMILA DE AZEVEDO,ASCHILEY CAMILA DE AZEVEDO,BRA-081-SAO PAULO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535,INC000001008022,MARCIO ROBERTO ALVES,BRA-078-PARA DE MINAS,Mobile Phone ITB - BR,BR_21 - Problema Telefonia Celular,Pendente,2 de mai de 2024 09:33:45,MARCIO ROBERTO ALVES,MARCIO ROBERTO ALVES,BRA-078-PARA DE MINAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,INC000000986933,Lucas Wisnieski,BRA-026-HUNGRIA,NEWCO_CONSULTORIA_COMPRAS - BR,BR_2024 - Newco,Pendente,11 de mar de 2024 17:42:20,Lucas Wisnieski,Lucas Wisnieski,BRA-026-HUNGRIA,...,NaN,NaN,NaN,NaN,NaN,NaN,Problemas Técnicos (Modulos SAP),Comercial Logistico (SD / LES ),Lucas Wisnieski,Erro na determinação do IVA na criação do pedi...


In [14]:
dia = datetime.today().strftime('%d.%m')
notificacao = Notification(app_id="Telefonia", title="Notificação",
                    msg="Relatorio da telefonia finalizado com sucesso",
                    duration="long")

notificacao.add_actions(label="Abrir", launch=rf"C:\Users\042006431\PycharmProjects\pythonProject1\Kace\Chamados_tel_{dia}.xlsx")

In [15]:
df_full.to_excel(f'Chamados_tel_{dia}.xlsx', index=False)
notificacao.show()